## 2013 play by play data

In [41]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sn
from sklearn.metrics import silhouette_samples, silhouette_score


sys.setrecursionlimit(100000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

### import 2013 nhl data set

In [42]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_pbp20132014.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_pbp20132014.csv')
da = da.drop('Unnamed: 0', axis=1)

- rename columns and create a season variable

In [43]:
da['Season'] = 2013
da = da.rename(columns={ 'Game_Id' : 'GameNumber', 'Ev_Zone': 'EventZone', 'Time_Elapsed': 'Time', 'Ev_Team': 'EventTeamCode', 'Description': 'EventDetail', 'Away_Team': 'ATeamCode', 'Home_Team':'HTeamCode', 'p1_name': 'EventPlayer1','p2_name': 'EventPlayer2', 'p3_name': 'EventPlayer3', 'awayPlayer1': 'APlayer1', 'awayPlayer2': 'APlayer2', 'awayPlayer3': 'APlayer3', 'awayPlayer4': 'APlayer4', 'awayPlayer5': 'APlayer5', 'awayPlayer6': 'APlayer6', 'homePlayer1': 'HPlayer1', 'homePlayer2': 'HPlayer2', 'homePlayer3': 'HPlayer3','homePlayer4': 'HPlayer4', 'homePlayer5': 'HPlayer5','homePlayer6': 'HPlayer6', 'Away_Score': 'AGoals', 'Home_Score': 'HGoals', 'Away_Goalie': 'AGoalie', 'Home_Goalie': 'HGoalie', 'Away_Players': 'PlayersA', 'Home_Players': 'PlayersH' })
da = da[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'Strength', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer1', 'APlayer2', 'APlayer3', 'APlayer4', 'APlayer5', 'APlayer6', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer1', 'HPlayer2', 'HPlayer3', 'HPlayer4', 'HPlayer5', 'HPlayer6', 'HGoalie', 'PlayersH']]
da = da.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- keep only regular season games and exclude irrelevant on-ice events

In [44]:
da = da[da['GameNumber'] <= 21320]
da = da[da['Period'] <= 4]
da = da[da['Period'] >= 1]
da = da[da['Event']!='STOP']
da = da[da['Event']!='EISTR']
da = da[da['Event']!='EIEND']
da = da[da['Event']!='PSTR']
da = da[da['Event']!='PEND']
da = da[da['Event']!='SOC']
da = da[da['Event']!='GEND']

- create an event number variable that will count the number of events per game. 
- create an advantage type variable for even strength, power play and short handed situations.

In [45]:
da['EventNumber'] = da.groupby(['Season', 'GameNumber']).cumcount()+1

In [46]:
da['AdvantageType'] = da.apply(lambda x: 'EV' if ((x['PlayersA'] == x['PlayersH']) & (x['PlayersA'] != 1) & (x['PlayersH'] != 1)) else 'PP' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'PP' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'SH' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'SH' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'PP' if ((x['PlayersA'] == 1 ) & (x['PlayersH'] == 1)) else np.nan, axis=1) 

In [47]:
da['AdvantageType'].value_counts()

EV    264338
PP     35520
SH     15933
Name: AdvantageType, dtype: int64

- reshape data from wide to long for away team players and home team players

In [48]:
db = da.copy()
a = [col for col in db.columns if 'APlayer' in col]
b = [col for col in db.columns if 'HPlayer' in col]
db = pd.lreshape(db, {'APlayer' : a, 'HPlayer' : b })
db.head()

,AGoalie,ATeamCode,AdvantageType,Date,Event,EventDetail,EventNumber,EventPlayer1,EventPlayer2,EventPlayer3,EventTeamCode,GameNumber,HGoalie,HTeamCode,Period,PlayersA,PlayersH,Season,Strength,Time,APlayer,HPlayer
0,JAMES REIMER,TOR,EV,2013-10-01,FAC,TOR won Neu. Zone - TOR #42 BOZAK vs MTL #14 P...,1,TYLER BOZAK,TOMAS PLEKANEC,NaN,TOR,20001,CAREY PRICE,MTL,1,6,6,2013,5x5,0:00,TYLER BOZAK,TOMAS PLEKANEC
1,JAMES REIMER,TOR,PP,2013-10-01,PENL,"TOR #3 PHANEUF Boarding(2 min), Def. Zone Draw...",2,DION PHANEUF,BRANDON PRUST,NaN,TOR,20001,CAREY PRICE,MTL,1,6,5,2013,4x5,10:03,TYLER BOZAK,LARS ELLER
2,JAMES REIMER,TOR,EV,2013-10-01,FAC,MTL won Off. Zone - TOR #42 BOZAK vs MTL #81 E...,3,LARS ELLER,TYLER BOZAK,NaN,MTL,20001,CAREY PRICE,MTL,1,5,5,2013,4x4,10:03,TYLER BOZAK,ALEX GALCHENYUK
3,JAMES REIMER,TOR,EV,2013-10-01,GOAL,"MTL #81 ELLER(1), Wrist, Off. Zone, 18 ft.Assi...",4,LARS ELLER,RAPHAEL DIAZ,ALEX GALCHENYUK,MTL,20001,CAREY PRICE,MTL,1,5,5,2013,4x4,10:08,TYLER BOZAK,ALEX GALCHENYUK
4,JAMES REIMER,TOR,EV,2013-10-01,FAC,TOR won Neu. Zone - TOR #63 BOLLAND vs MTL #81...,5,DAVE BOLLAND,LARS ELLER,NaN,TOR,20001,CAREY PRICE,MTL,1,5,5,2013,4x4,10:08,DAVE BOLLAND,ALEX GALCHENYUK


In [49]:
db = db[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'AdvantageType', 'Strength', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer', 'HGoalie', 'PlayersH']]
db = db.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- reshape data from wide to long for away and home team code and player respectfully.

In [50]:
dc = db.copy()
dc = dc.rename(columns={'EventTeamCode': 'EventTeam', 'EventPlayer1': 'EventP1', 'EventPlayer2': 'EventP2', 'EventPlayer3': 'EventP3', 'PlayersA':'PA', 'PlayersH': 'PH'})

In [51]:
a = [col for col in dc.columns if 'Code' in col]
b = [col for col in dc.columns if 'Player' in col]
c = [col for col in dc.columns if 'Goalie' in col]
d = [col for col in dc.columns if 'Position' in col]
dc = pd.lreshape(dc, {'TeamCode' : a, 'PlayerName' : b, 'Goalie': c})

In [52]:
dc = dc.rename(columns={'EventTeam': 'EventTeamCode', 'EventP1': 'EventPlayer1', 'EventP2': 'EventPlayer2', 'EventP3': 'EventPlayer3', 'PA':'PlayersA', 'PH': 'PlayersH'})

In [53]:
dc.shape

(3627691, 19)

### games played

- calculate the games each player participated in for the duration of the 2011 regular season.

In [54]:
s = dc.copy()
s = s[['Season', 'GameNumber', 'TeamCode', 'PlayerName']]
s = s.sort_values(['Season', 'GameNumber', 'TeamCode'], ascending=[True, True, True])
s = s.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [55]:
s['GP'] = s.groupby(['Season', 'TeamCode','PlayerName'])['GameNumber'].transform('count')
s = s.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
s = s[['Season', 'TeamCode', 'PlayerName', 'GP']]
s.head()

,Season,TeamCode,PlayerName,GP
1814260,2013,MTL,TOMAS PLEKANEC,81
1814261,2013,MTL,RENE BOURQUE,63
1814262,2013,MTL,BRIAN GIONTA,81
1814263,2013,MTL,JOSH GORGES,66
1814264,2013,MTL,PK SUBBAN,82


In [56]:
s.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
dtype: int64

In [57]:
s.shape

(1056, 4)

### shots for and against

- calculate the shots against each player faced througout the duration of a season. 
- create shot for and shot against variables. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [71]:
sa = dc.copy()
sa = sa[sa['Event'].isin(['SHOT','GOAL'])]

In [72]:
sa['ShotF'] = sa.apply(lambda x: 1 if x['EventTeamCode'] == x['TeamCode'] else 0, axis=1)
sa['ShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotF'].transform('sum')

sa['ShotA'] = sa.apply(lambda x: 1 if x['EventTeamCode'] != x['TeamCode'] else 0, axis=1)
sa['ShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotA'].transform('sum')

- create shot for and shot against variables for even strength situations only. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [73]:
sa['EVShotF'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
sa['EVShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotF'].transform('sum')

sa['EVShotA'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)
sa['EVShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotA'].transform('sum')

- calculate the shot differential for all shots and even strength shots.

In [74]:
sa['DShots'] = sa['ShotsF'] - sa['ShotsA']
sa['EVDShots'] = sa['EVShotsF'] - sa['EVShotsA']

In [75]:
sa = sa [['Season', 'TeamCode', 'PlayerName', 'ShotsF', 'ShotsA', 'DShots', 'EVShotsF', 'EVShotsA', 'EVDShots' ]]
sa = sa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
sa.head()

,Season,TeamCode,PlayerName,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots
16,2013,TOR,TYLER BOZAK,656,710,-54,502,633,-131
18,2013,TOR,PHIL KESSEL,985,916,69,730,915,-185
20,2013,TOR,PAUL RANGER,376,577,-201,364,479,-115
22,2013,TOR,JAKE GARDINER,834,887,-53,680,838,-158
24,2013,TOR,JAMES REIMER,673,1067,-394,673,902,-229


In [76]:
sa.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
dtype: int64

In [77]:
sa.shape

(1056, 9)

### goals for and against

- create goals for and goals against variable. If event team code is the same as team code assign a value of 1 for goals for and a value of 0 for goals against. If event team code is different to team code, assign a value of 0 for golas for and a value of 1 for goals against. Display total goals for and against per player for a season.

In [78]:
gfa = dc.copy()
gfa = gfa[gfa['Event'] == 'GOAL']

In [79]:
gfa['EVGoalF'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
gfa['EVGoalA'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)

In [80]:
gfa['GoalF'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] == x['TeamCode']) else 0, axis=1)
gfa['GoalA'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] != x['TeamCode']) else 0, axis=1)

In [81]:
gfa['EVGoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalF'].transform('sum')
gfa['EVGoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalA'].transform('sum')
gfa['GoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalF'].transform('sum')
gfa['GoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalA'].transform('sum')

In [82]:
gfa = gfa[['Season', 'GameNumber', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA']]
gfa = gfa.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
gfa['Plus/Minus'] = gfa['EVGoalsF'] - gfa['EVGoalsA']
gfa = gfa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
gfa = gfa[['Season', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA', 'Plus/Minus']]
gfa.head()

,Season,TeamCode,PlayerName,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus
16,2013,TOR,TYLER BOZAK,77,65,59,58,1
18,2013,TOR,PHIL KESSEL,101,71,73,71,2
20,2013,TOR,PAUL RANGER,35,44,32,33,-1
22,2013,TOR,JAKE GARDINER,72,56,51,51,0
24,2013,TOR,JAMES REIMER,45,96,45,70,-25


In [83]:
gfa.shape

(1023, 8)

### create dataframes for all on-ice event types

#### goals dataframe (dg)

In [84]:
dg = dc.copy()
dg = dg[dg['Event'] == 'GOAL']

In [85]:
dg = dg[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dg = dg.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a goal variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [86]:
dg['Goal'] = dg.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dg = dg.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dg['Goals'] = dg.groupby(['Season','TeamCode', 'PlayerName'])['Goal'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [87]:
dg = dg.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dg = dg.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dg = dg[['Season', 'TeamCode', 'PlayerName', 'Goals']]
dg.head()

,Season,TeamCode,PlayerName,Goals
16,2013,MTL,LARS ELLER,12
97,2013,MTL,BRENDAN GALLAGHER,19
598,2013,TOR,JAMES VAN RIEMSDYK,30
821,2013,TOR,TYLER BOZAK,19
1067,2013,TOR,DION PHANEUF,8


In [88]:
dg.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Goals         0
dtype: int64

In [89]:
dg.shape

(736, 4)

#### assists dataframe (dast)

In [90]:
dast = dc.copy()
dast = dast[dast['Event'] == 'GOAL']

In [91]:
dast = dast[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer2', 'EventPlayer3']]
dast = dast.rename(columns={'EventPlayer2': 'Assist1Name', 'EventPlayer3': 'Assist2Name', 'EventTeamCode':'TeamCode'})

- reshape assist players 1 and 2 to create one column of assist players

In [92]:
a = [col for col in dast.columns if 'Name' in col]
dast = pd.lreshape(dast, {'PlayerName' : a})

- create an assist variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [93]:
dast['Assist'] = dast.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dast = dast.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dast['Assists'] = dast.groupby(['Season','TeamCode', 'PlayerName'])['Assist'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [94]:
dast = dast.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dast = dast.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dast = dast[['Season', 'TeamCode', 'PlayerName', 'Assists']]
dast.head()

,Season,TeamCode,PlayerName,Assists
0,2013,MTL,RAPHAEL DIAZ,11
5,2013,MTL,LARS ELLER,14
11,2013,TOR,PHIL KESSEL,43
15,2013,TOR,NAZEM KADRI,30
21,2013,TOR,TROY BODIE,7


In [95]:
dast.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Assists       0
dtype: int64

In [96]:
dast.shape

(831, 4)

#### shot dataframe (ds)

In [97]:
ds = dc.copy()
ds = ds[ds['Event'] == 'SHOT']

In [98]:
ds = ds[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
ds = ds.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})
ds.head()

,Season,GameNumber,EventNumber,Event,TeamCode,PlayerName
31,2013,20001,7,SHOT,TOR,DAVE BOLLAND
32,2013,20001,7,SHOT,TOR,DAVE BOLLAND
33,2013,20001,7,SHOT,TOR,DAVE BOLLAND
34,2013,20001,7,SHOT,TOR,DAVE BOLLAND
35,2013,20001,7,SHOT,TOR,DAVE BOLLAND


- create a shot variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [99]:
ds['Shot'] = ds.apply(lambda x: 1 if (x['Event'] == 'SHOT') else 0, axis=1)
ds = ds.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
ds['Shots'] = ds.groupby(['Season','TeamCode', 'PlayerName'])['Shot'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [100]:
ds = ds.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
ds = ds.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
ds = ds[['Season', 'TeamCode', 'PlayerName', 'Shots']]
ds.head()

,Season,TeamCode,PlayerName,Shots
31,2013,TOR,DAVE BOLLAND,26
46,2013,MTL,PK SUBBAN,194
109,2013,TOR,PHIL KESSEL,268
126,2013,MTL,RENE BOURQUE,109
136,2013,TOR,CODY FRANSON,110


In [101]:
ds.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Shots         0
dtype: int64

In [102]:
ds.shape

(924, 4)

#### block dataframe (db)

In [103]:
dbl = dc.copy()
dbl = dbl[dbl['Event'] == 'BLOCK']

In [104]:
dbl.shape

(397937, 19)

In [105]:
dbl['EventPlayer2'].isnull().sum()

0

In [106]:
dbl['EventPlayer3'].isnull().sum()

397937

In [107]:
dbl['BlockTeam'] = dbl.EventDetail.str.split(' ', expand = True)[6]
dbl['BlockTeamCode'] = dbl.BlockTeam.str.split(' ', expand = True)[0]

In [108]:
dbl = dbl[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventPlayer1', 'BlockTeamCode']]
dbl = dbl.rename(columns={'EventPlayer1': 'PlayerName', 'BlockTeamCode':'TeamCode'})

- create a block variable. If event type is a block assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [109]:
dbl['Block'] = dbl.apply(lambda x: 1 if (x['Event'] == 'BLOCK') else 0, axis=1)
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dbl['Blocks'] = dbl.groupby(['Season','TeamCode', 'PlayerName'])['Block'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [110]:
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dbl = dbl.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dbl = dbl[['Season', 'TeamCode', 'PlayerName', 'Blocks']]
dbl.head()

,Season,TeamCode,PlayerName,Blocks
146,2013,TOR,MARK FRASER,51
181,2013,TOR,JAKE GARDINER,86
241,2013,TOR,TYLER BOZAK,41
277,2013,MTL,JOSH GORGES,175
289,2013,MTL,RAPHAEL DIAZ,99


In [111]:
dbl.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Blocks        0
dtype: int64

In [112]:
dbl.shape

(1104, 4)

#### hit dataframe (dh)

In [114]:
dhit = dc.copy()
dhit = dhit[dhit['Event'] == 'HIT']

In [115]:
dhit.shape

(682051, 19)

In [116]:
dhit['EventPlayer2'].isnull().sum()

0

In [117]:
dhit['EventPlayer3'].isnull().sum()

682051

In [118]:
dhit = dhit[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dhit = dhit.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a hit variable. If event type is a hit assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [119]:
dhit['Hit'] = dhit.apply(lambda x: 1 if (x['Event'] == 'HIT') else 0, axis=1)
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dhit['Hits'] = dhit.groupby(['Season','TeamCode', 'PlayerName'])['Hit'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [120]:
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dhit = dhit.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dhit = dhit[['Season', 'TeamCode', 'PlayerName', 'Hits']]
dhit.head()

,Season,TeamCode,PlayerName,Hits
36,2013,TOR,DAVE BOLLAND,38
41,2013,TOR,NIKOLAI KULEMIN,153
61,2013,TOR,COLTON ORR,112
193,2013,TOR,MASON RAYMOND,38
205,2013,TOR,TYLER BOZAK,62


In [121]:
dhit.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Hits          0
dtype: int64

In [122]:
dhit.shape

(959, 4)

#### penalty dataframe (dpen)

In [123]:
dp = dc.copy()
dp = dp[dp['Event'] == 'PENL']

- use event detail to find the duration of a given penalty and to assign it to the proper player. Major penalty is 5 minutes, so a value of 5 is assigned for every event that had a major penalty.

In [124]:
dp['Pen'] = dp.EventDetail.str.split(')', expand = True)[0]
dp['Penal'] = dp.Pen.str.split('(', expand = True)[1]
dp['Penalt'] = dp.Penal.str.split(' ', expand = True)[0]
dp['Penalty'] = dp.apply(lambda x: 5 if x['Penalt'] == 'maj' else x['Penalt'], axis=1)
dp['Penalty'] = dp['Penalty'].convert_objects(convert_numeric=True)
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


- use event detail to find penalty team code and penalty player name.

In [125]:
dp['PenaltyTeamCode'] = dp.EventDetail.str.split('\\s', expand = True)[0]
dp['PN'] = dp.EventDetail.str.split('\\s', expand = True)[1]
dp['PNumber'] = dp.PN.str.split('#', expand = True)[1]
dp['PenaltyPlayerNumber'] = dp.PNumber.str.split(' ', expand = True)[0]
dp['PenaltyPlayerLName'] = dp.EventDetail.str.split('\\s', expand = True)[2]
#dp['PenaltyPlayerLName'] = dp.PPlLN.str.split(' ', expand = True)[0]
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName']]

- seperate player name into first and last. The purpose is to connect the correct player name to penalty player name.

In [126]:
dp['PlayerFName'] = dp.PlayerName.str.split('\\s', expand = True)[0]
dp['PlayerLName'] = dp.PlayerName.str.split('\\s', expand = True)[1]
dp['PenaltyPlayerFName'] = dp.apply(lambda x: x['PlayerFName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName'])) else np.nan, axis=1)
dp['PenaltyPlayerFName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyPlayerFName'].apply(lambda x: x.ffill().bfill())

In [127]:
dp['PenaltyName'] = dp.apply(lambda x: x['PlayerName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName']) & (x['PlayerFName'] == x['PenaltyPlayerFName'])) else np.nan, axis=1)
dp['PenaltyName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerLName', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyName'].apply(lambda x: x.ffill().bfill())


- drop duplicates per game and event number. Calculate the total number of penalties for each player.

In [128]:
dp = dp.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])
dp['Penalties'] = dp.groupby(['Season', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])['Penalty'].transform('sum')
dp['Penalties'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PlayerName'])['Penalties'].apply(lambda x: x.ffill().bfill())
#dp.head()

- drop duplicates to keep one observation per player for the season.

In [129]:
dp = dp[['Season', 'PenaltyTeamCode', 'PenaltyName', 'Penalties']]
dp = dp.drop_duplicates(['Season', 'PenaltyTeamCode', 'PenaltyName'])
dp = dp.rename(columns={'PenaltyTeamCode':'TeamCode', 'PenaltyName':'PlayerName'})
dp = dp[['Season', 'TeamCode', 'PlayerName', 'Penalties']]
dp.head()

,Season,TeamCode,PlayerName,Penalties
6,2013,TOR,DION PHANEUF,140.0
115,2013,TOR,JAMES VAN RIEMSDYK,50.0
559,2013,MTL,LARS ELLER,68.0
585,2013,MTL,TOMAS PLEKANEC,38.0
645,2013,MTL,NaN,NaN


- keep observations that are not null. Since a team can be allocated a penalty, those observations are excluded from the data.

In [130]:
dp['PlayerName'].isnull().sum()

30

In [131]:
dp = dp[dp['PlayerName'].notnull()]

In [132]:
dp.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Penalties     0
dtype: int64

In [133]:
dp.shape

(872, 4)

####  miss dataframe (dmiss)

In [134]:
dmiss = dc.copy()
dmiss = dmiss[dmiss['Event'] == 'MISS']

In [135]:
dmiss.shape

(329252, 19)

In [136]:
dmiss['EventPlayer2'].isnull().sum()

329252

In [137]:
dmiss['EventPlayer3'].isnull().sum()

329252

In [138]:
dmiss = dmiss[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dmiss = dmiss.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a miss variable. If event type is a miss assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [139]:
dmiss['Miss'] = dmiss.apply(lambda x: 1 if (x['Event'] == 'MISS') else 0, axis=1)
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dmiss['Misses'] = dmiss.groupby(['Season','TeamCode', 'PlayerName'])['Miss'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [140]:
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dmiss = dmiss.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dmiss = dmiss[['Season', 'TeamCode', 'PlayerName', 'Misses']]
dmiss.head()

,Season,TeamCode,PlayerName,Misses
67,2013,MTL,GEORGE PARROS,1
175,2013,MTL,RENE BOURQUE,54
217,2013,MTL,TRAVIS MOEN,29
265,2013,TOR,JAMES VAN RIEMSDYK,72
301,2013,TOR,CODY FRANSON,59


In [141]:
dmiss.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Misses        0
dtype: int64

In [142]:
dmiss.shape

(890, 4)

####  takeaway dataframe (dtake)

In [143]:
dtake = dc.copy()
dtake = dtake[dtake['Event'] == 'TAKE']

In [144]:
dtake.shape

(186893, 19)

In [145]:
dtake['EventPlayer2'].isnull().sum()

186893

In [146]:
dtake['EventPlayer3'].isnull().sum()

186893

In [147]:
dtake = dtake[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dtake = dtake.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a takeaway variable. If event type is a takeaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [148]:
dtake['Takeaway'] = dtake.apply(lambda x: 1 if (x['Event'] == 'TAKE') else 0, axis=1)
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dtake['Takeaways'] = dtake.groupby(['Season','TeamCode', 'PlayerName'])['Takeaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [149]:
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dtake = dtake.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dtake = dtake[['Season', 'TeamCode', 'PlayerName', 'Takeaways']]
dtake.head()

,Season,TeamCode,PlayerName,Takeaways
131,2013,TOR,DAVE BOLLAND,10
151,2013,MTL,DANNY BRIERE,22
666,2013,MTL,JOSH GORGES,11
816,2013,TOR,TYLER BOZAK,49
874,2013,TOR,JOFFREY LUPUL,25


In [150]:
dtake.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Takeaways     0
dtype: int64

In [151]:
dtake.shape

(858, 4)

####  giveaway dataframe (dgive)

In [152]:
dgive = dc.copy()
dgive = dgive[dgive['Event'] == 'GIVE']

In [153]:
dgive.shape

(223653, 19)

In [154]:
dgive['EventPlayer2'].isnull().sum()

223653

In [155]:
dgive['EventPlayer3'].isnull().sum()

223653

In [156]:
dgive = dgive[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dgive = dgive.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a giveaway variable. If event type is a giveaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [157]:
dgive['Giveaway'] = dgive.apply(lambda x: 1 if (x['Event'] == 'GIVE') else 0, axis=1)
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dgive['Giveaways'] = dgive.groupby(['Season','TeamCode', 'PlayerName'])['Giveaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [158]:
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dgive = dgive.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dgive = dgive[['Season', 'TeamCode', 'PlayerName', 'Giveaways']]
dgive.head()

,Season,TeamCode,PlayerName,Giveaways
141,2013,TOR,CODY FRANSON,68
187,2013,MTL,ANDREI MARKOV,82
337,2013,MTL,TRAVIS MOEN,11
457,2013,TOR,MARK FRASER,16
607,2013,MTL,CAREY PRICE,58


In [159]:
dgive.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Giveaways     0
dtype: int64

In [160]:
dgive.shape

(946, 4)

### import 2013 player shifts

In [161]:
dshift = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_shifts20132014.csv')
#dshift = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_shifts20132014.csv')
dshift = dshift.drop('Unnamed: 0', axis=1)

- rename variables, keep only regular season games and create a season variable.

In [162]:
dshift['Season'] = 2013
dshift= dshift.rename(columns={ 'Game_Id' : 'GameNumber', 'Team': 'TeamCode', 'Player': 'PlayerName' })
dshift = dshift[dshift['GameNumber'] <= 21230]
dshift = dshift[['Season', 'Date', 'GameNumber', 'Period', 'TeamCode', 'PlayerName', 'Player_Id', 'Start', 'End', 'Duration']]
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration
0,2013,2013-10-01,20001,1,MTL,TOMAS PLEKANEC,8469521.0,0.0,29.0,29.0
1,2013,2013-10-01,20001,1,MTL,BRIAN GIONTA,8467407.0,0.0,34.0,34.0
2,2013,2013-10-01,20001,1,MTL,RENE BOURQUE,8471504.0,0.0,34.0,34.0
3,2013,2013-10-01,20001,1,TOR,JAMES VAN RIEMSDYK,8474037.0,0.0,36.0,36.0
4,2013,2013-10-01,20001,1,TOR,PHIL KESSEL,8473548.0,0.0,36.0,36.0


- calcuate the seconds an individual played per game. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [163]:
dshift['Seconds'] = dshift.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])['Duration'].transform('sum')
dshift['Minutes'] = dshift['Seconds']/60
dshift['Minutes'] = dshift['Minutes'].round(2)

In [164]:
dshift = dshift.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [165]:
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes
0,2013,2013-10-01,20001,1,MTL,TOMAS PLEKANEC,8469521.0,0.0,29.0,29.0,1086.0,18.10
1,2013,2013-10-01,20001,1,MTL,BRIAN GIONTA,8467407.0,0.0,34.0,34.0,921.0,15.35
2,2013,2013-10-01,20001,1,MTL,RENE BOURQUE,8471504.0,0.0,34.0,34.0,897.0,14.95
3,2013,2013-10-01,20001,1,TOR,JAMES VAN RIEMSDYK,8474037.0,0.0,36.0,36.0,1260.0,21.00
4,2013,2013-10-01,20001,1,TOR,PHIL KESSEL,8473548.0,0.0,36.0,36.0,1271.0,21.18


- calcuate the seconds an individual played for the whole regular season. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [166]:
dshift['TSeconds'] = dshift.groupby(['Season', 'TeamCode', 'PlayerName'])['Seconds'].transform('sum')
dshift['TOI'] = dshift['TSeconds']/60
dshift['TOI'] = dshift['TOI'].round(0)
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes,TSeconds,TOI
0,2013,2013-10-01,20001,1,MTL,TOMAS PLEKANEC,8469521.0,0.0,29.0,29.0,1086.0,18.10,96107.0,1602.0
1,2013,2013-10-01,20001,1,MTL,BRIAN GIONTA,8467407.0,0.0,34.0,34.0,921.0,15.35,86731.0,1446.0
2,2013,2013-10-01,20001,1,MTL,RENE BOURQUE,8471504.0,0.0,34.0,34.0,897.0,14.95,53613.0,894.0
3,2013,2013-10-01,20001,1,TOR,JAMES VAN RIEMSDYK,8474037.0,0.0,36.0,36.0,1260.0,21.00,101071.0,1685.0
4,2013,2013-10-01,20001,1,TOR,PHIL KESSEL,8473548.0,0.0,36.0,36.0,1271.0,21.18,101678.0,1695.0


- keep only observation per player for the season

In [167]:
dshift = dshift.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dshift = dshift[['Season', 'TeamCode', 'PlayerName', 'TOI']]
dshift.head()

,Season,TeamCode,PlayerName,TOI
0,2013,MTL,TOMAS PLEKANEC,1602.0
1,2013,MTL,BRIAN GIONTA,1446.0
2,2013,MTL,RENE BOURQUE,894.0
3,2013,TOR,JAMES VAN RIEMSDYK,1685.0
4,2013,TOR,PHIL KESSEL,1695.0


In [168]:
dshift.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
TOI           0
dtype: int64

In [169]:
dshift.shape

(1056, 4)

### import 2012 player positions

In [182]:
dpos = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_positions20132014.csv')
#dpos = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_positions20132014.csv')

In [183]:
dpos.head()

,FirstName,LastName,TeamCode,Position,FName,LName,PlayerName
0,Spencer,Abbott,TOR,LW/RW,SPENCER,ABBOTT,SPENCER ABBOTT
1,Justin,Abdelkader,DET,LW/C,JUSTIN,ABDELKADER,JUSTIN ABDELKADER
2,Will,Acton,EDM,C/RW,WILL,ACTON,WILL ACTON
3,Luke,Adam,BUF,C/LW,LUKE,ADAM,LUKE ADAM
4,Craig,Adams,PIT,RW,CRAIG,ADAMS,CRAIG ADAMS


In [184]:
dpos['Season'] = 2013
dpos= dpos.rename(columns={ 'Team': 'TeamCode', 'Pos ': 'Position'})
dpos = dpos[['Season', 'TeamCode', 'PlayerName', 'Position']]
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2013,TOR,SPENCER ABBOTT,LW/RW
1,2013,DET,JUSTIN ABDELKADER,LW/C
2,2013,EDM,WILL ACTON,C/RW
3,2013,BUF,LUKE ADAM,C/LW
4,2013,PIT,CRAIG ADAMS,RW


In [185]:
dpos.dtypes

Season         int64
TeamCode      object
PlayerName    object
Position      object
dtype: object

In [186]:
dpos['Position'] = dpos.apply(lambda x: 'C' if x['Position'] == 'C/LW' else 'C' if x['Position'] == 'C/RW' else 'C' if x['Position'] == 'C/RW/LW' else 'C' if x['Position'] == 'C/LW/RW' else 'W' if x['Position'] == 'RW' else 'W' if x['Position'] == 'LW' else 'W' if x['Position'] == 'LW/C' else 'W' if x['Position'] == 'RW/C' else 'W' if x['Position'] == 'RW/LW' else 'W' if x['Position'] == 'LW/RW' else 'W' if x['Position'] == 'RW/LW/C' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'LW/D' else 'W' if x['Position'] == 'LW/C/RW' else 'W' if x['Position'] == 'LW/RW/C' else 'D' if x['Position'] == 'D/LW' else 'D' if x['Position'] == 'D/RW' else 'C' if x['Position'] == 'C/D' else 'W' if x['Position'] == 'RW/D' else x['Position'] , axis=1)
dpos['Position'].value_counts()

W    328
D    303
C    255
G     97
Name: Position, dtype: int64

In [187]:
dpos['TeamCode'] = dpos.apply(lambda x: 'L.A' if x['TeamCode'] == 'LAK' else 'N.J' if x['TeamCode'] == 'NJD' else 'S.J' if x['TeamCode'] == 'SJS' else 'T.B' if x['TeamCode'] == 'TBL' else x['TeamCode'] , axis=1)
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2013,TOR,SPENCER ABBOTT,W
1,2013,DET,JUSTIN ABDELKADER,W
2,2013,EDM,WILL ACTON,C
3,2013,BUF,LUKE ADAM,C
4,2013,PIT,CRAIG ADAMS,W


In [188]:
dpos.shape

(983, 4)

### merge data frames

- merge games played, goals and assists.

In [193]:
dd = pd.merge(s, dpos, on=['Season', 'TeamCode', 'PlayerName'], how='left').merge(dg,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dast,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')

- forward and backward fill players that have been traded during the season. Fill in positions for three players.

In [194]:
dd['Position'] = dd.groupby(['PlayerName'])['Position'].apply(lambda x: x.ffill().bfill())
dd['Position'] = dd.apply(lambda x: 'D' if x['PlayerName'] == 'MICHAEL KOSTKA' else 'C' if x['PlayerName'] == 'DANNY BRIERE' else x['Position'] , axis=1)

dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists
0,2013,MTL,TOMAS PLEKANEC,81,C,20.0,23.0
1,2013,MTL,RENE BOURQUE,63,W,9.0,7.0
2,2013,MTL,BRIAN GIONTA,81,W,18.0,22.0
3,2013,MTL,JOSH GORGES,66,D,1.0,13.0
4,2013,MTL,PK SUBBAN,82,D,10.0,43.0


In [195]:
dd.isnull().sum()

Season          0
TeamCode        0
PlayerName      0
GP              0
Position        0
Goals         320
Assists       225
dtype: int64

In [196]:
fix = dd.copy()
fix = fix[fix['Position'].isnull()]
fix.head(20)

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists


- fill in goals and assists with 0 for players who had no points.

In [197]:
dd['Goals'] = dd['Goals'].fillna(0)
dd['Assists'] = dd['Assists'].fillna(0)
dd['Points'] = dd['Goals'] + dd['Assists']
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points
0,2013,MTL,TOMAS PLEKANEC,81,C,20.0,23.0,43.0
1,2013,MTL,RENE BOURQUE,63,W,9.0,7.0,16.0
2,2013,MTL,BRIAN GIONTA,81,W,18.0,22.0,40.0
3,2013,MTL,JOSH GORGES,66,D,1.0,13.0,14.0
4,2013,MTL,PK SUBBAN,82,D,10.0,43.0,53.0


In [198]:
dd.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
dtype: int64

In [199]:
de = pd.merge(dd, gfa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(ds,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(sa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dbl,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dhit,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dp,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dtake,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dgive,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dmiss,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dshift,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')
de.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus,Shots,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots,Blocks,Hits,Penalties,Takeaways,Giveaways,Misses,TOI
0,2013,MTL,TOMAS PLEKANEC,81,C,20.0,23.0,43.0,61.0,56.0,46.0,36.0,10.0,180.0,699,860,-161,577,650,-73,43.0,44.0,38.0,38.0,37.0,91.0,1602.0
1,2013,MTL,RENE BOURQUE,63,W,9.0,7.0,16.0,28.0,21.0,20.0,21.0,-1.0,109.0,398,433,-35,340,430,-90,15.0,90.0,32.0,13.0,9.0,54.0,894.0
2,2013,MTL,BRIAN GIONTA,81,W,18.0,22.0,40.0,50.0,47.0,38.0,39.0,-1.0,166.0,634,759,-125,541,652,-111,52.0,46.0,20.0,38.0,21.0,81.0,1446.0
3,2013,MTL,JOSH GORGES,66,D,1.0,13.0,14.0,44.0,55.0,43.0,37.0,6.0,34.0,527,736,-209,510,562,-52,175.0,97.0,12.0,11.0,29.0,37.0,1402.0
4,2013,MTL,PK SUBBAN,82,D,10.0,43.0,53.0,111.0,79.0,70.0,74.0,-4.0,194.0,1115,920,195,792,882,-90,124.0,135.0,81.0,26.0,83.0,82.0,2019.0


- fill in with 0 for all nan values since not all players have been on the ice for a goal for or a goal against. 

In [200]:
de['GoalsF'] = de['GoalsF'].fillna(0)
de['GoalsA'] = de['GoalsA'].fillna(0)
de['EVGoalsF'] = de['EVGoalsF'].fillna(0)
de['EVGoalsA'] = de['EVGoalsA'].fillna(0)
de['Plus/Minus'] = de['Plus/Minus'].fillna(0)
de['Shots'] = de['Shots'].fillna(0)
de['Hits'] = de['Hits'].fillna(0)
de['Blocks'] = de['Blocks'].fillna(0)
de['Penalties'] = de['Penalties'].fillna(0)
de['Giveaways'] = de['Giveaways'].fillna(0)
de['Takeaways'] = de['Takeaways'].fillna(0)
de['Misses'] = de['Misses'].fillna(0)

In [201]:
de.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
GoalsF        0
GoalsA        0
EVGoalsF      0
EVGoalsA      0
Plus/Minus    0
Shots         0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
Blocks        0
Hits          0
Penalties     0
Takeaways     0
Giveaways     0
Misses        0
TOI           0
dtype: int64

### total player stats for the season

- players that have been traded need to be evaluated for their season performance. Group by sesaon and player name to calcute the sum of each stat. 

In [202]:
de['TGP'] = de.groupby(['Season', 'PlayerName'])['GP'].transform('sum')
de['TGoals'] = de.groupby(['Season', 'PlayerName'])['Goals'].transform('sum')
de['TAssists'] = de.groupby(['Season', 'PlayerName'])['Assists'].transform('sum')
de['TPoints'] = de.groupby(['Season', 'PlayerName'])['Points'].transform('sum')
de['TGoalsF'] = de.groupby(['Season', 'PlayerName'])['GoalsF'].transform('sum')
de['TGoalsA'] = de.groupby(['Season', 'PlayerName'])['GoalsA'].transform('sum')
de['TEVGoalsF'] = de.groupby(['Season', 'PlayerName'])['EVGoalsF'].transform('sum')
de['TEVGoalsA'] = de.groupby(['Season', 'PlayerName'])['EVGoalsA'].transform('sum')
de['TPlus/Minus'] = de.groupby(['Season', 'PlayerName'])['Plus/Minus'].transform('sum')
de['TShots'] = de.groupby(['Season', 'PlayerName'])['Shots'].transform('sum')
de['TShotsF'] = de.groupby(['Season', 'PlayerName'])['ShotsF'].transform('sum')
de['TShotsA'] = de.groupby(['Season', 'PlayerName'])['ShotsA'].transform('sum')
de['TDShots'] = de.groupby(['Season', 'PlayerName'])['DShots'].transform('sum')
de['TEVShotsF'] = de.groupby(['Season', 'PlayerName'])['EVShotsF'].transform('sum')
de['TEVShotsA'] = de.groupby(['Season', 'PlayerName'])['EVShotsA'].transform('sum')
de['TEVDShots'] = de.groupby(['Season', 'PlayerName'])['EVDShots'].transform('sum')
de['TBlocks'] = de.groupby(['Season', 'PlayerName'])['Blocks'].transform('sum')
de['THits'] = de.groupby(['Season', 'PlayerName'])['Hits'].transform('sum')
de['TPenalties'] = de.groupby(['Season', 'PlayerName'])['Penalties'].transform('sum')
de['TTakeaways'] = de.groupby(['Season', 'PlayerName'])['Takeaways'].transform('sum')
de['TGiveaways'] = de.groupby(['Season', 'PlayerName'])['Giveaways'].transform('sum')
de['TMisses'] = de.groupby(['Season', 'PlayerName'])['Misses'].transform('sum')
de['TTOI'] = de.groupby(['Season', 'PlayerName'])['TOI'].transform('sum')

- divide all stats with total time on ice to scale the data for cluster analysis

In [203]:
de['TOIGoals'] = de['TGoals'] / de['TTOI']
de['TOIAssists'] = de['TAssists'] / de['TTOI']
de['TOIPoints'] = de['TPoints'] / de['TTOI']
de['TOIGoalsF'] = de['TGoalsF'] / de['TTOI']
de['TOIGoalsA'] = de['TGoalsA'] / de['TTOI']
de['TOIEVGoalsF'] = de['TEVGoalsF'] / de['TTOI']
de['TOIEVGoalsA'] = de['TEVGoalsA'] / de['TTOI']
de['TOIPlus/Minus'] = de['TPlus/Minus'] / de['TTOI']
de['TOIShots'] = de['TShots'] / de['TTOI']
de['TOIShotsF'] = de['TShotsF'] / de['TTOI']
de['TOIShotsA'] = de['TShotsA'] / de['TTOI']
de['TOIDShots'] = de['TDShots'] / de['TTOI']
de['TOIEVShotsF'] = de['TEVShotsF'] / de['TTOI']
de['TOIEVShotsA'] = de['TEVShotsA'] / de['TTOI']
de['TOIEVDShots'] = de['TEVDShots'] / de['TTOI']
de['TOIBlocks'] = de['TBlocks'] / de['TTOI']
de['TOIHits'] = de['THits'] / de['TTOI']
de['TOIPenalties'] = de['TPenalties'] / de['TTOI']
de['TOITakeaways'] = de['TTakeaways'] / de['TTOI']
de['TOIGiveaways'] = de['TGiveaways'] / de['TTOI']
de['TOIMisses'] = de['TMisses'] / de['TTOI']

- keep sesaon total variables and scaled variables

In [204]:
de = de[['Season', 'TeamCode', 'PlayerName', 'Position', 'TGP', 'TGoals', 'TAssists', 'TPoints', 'TGoalsF', 'TGoalsA', 'TEVGoalsF', 'TEVGoalsA', 'TPlus/Minus', 'TShots', 'TShotsF', 'TShotsA', 'TDShots', 'TEVShotsF', 'TEVShotsA', 'TEVDShots', 'TBlocks', 'THits', 'TPenalties', 'TTakeaways', 'TGiveaways', 'TMisses', 'TTOI', 'TOIGoals', 'TOIAssists', 'TOIPoints', 'TOIGoalsF', 'TOIGoalsA', 'TOIEVGoalsF', 'TOIEVGoalsA', 'TOIPlus/Minus', 'TOIShots', 'TOIShotsF', 'TOIShotsA', 'TOIDShots', 'TOIEVShotsF', 'TOIEVShotsA', 'TOIEVDShots', 'TOIBlocks', 'TOIHits', 'TOIPenalties', 'TOITakeaways', 'TOIGiveaways', 'TOIMisses']]
de.head()

,Season,TeamCode,PlayerName,Position,TGP,TGoals,TAssists,TPoints,TGoalsF,TGoalsA,TEVGoalsF,TEVGoalsA,TPlus/Minus,TShots,TShotsF,TShotsA,TDShots,TEVShotsF,TEVShotsA,TEVDShots,TBlocks,THits,TPenalties,TTakeaways,TGiveaways,TMisses,TTOI,TOIGoals,TOIAssists,TOIPoints,TOIGoalsF,TOIGoalsA,TOIEVGoalsF,TOIEVGoalsA,TOIPlus/Minus,TOIShots,TOIShotsF,TOIShotsA,TOIDShots,TOIEVShotsF,TOIEVShotsA,TOIEVDShots,TOIBlocks,TOIHits,TOIPenalties,TOITakeaways,TOIGiveaways,TOIMisses
0,2013,MTL,TOMAS PLEKANEC,C,81,20.0,23.0,43.0,61.0,56.0,46.0,36.0,10.0,180.0,699,860,-161,577,650,-73,43.0,44.0,38.0,38.0,37.0,91.0,1602.0,0.012484,0.014357,0.026841,0.038077,0.034956,0.028714,0.022472,0.006242,0.112360,0.436330,0.536829,-0.100499,0.360175,0.405743,-0.045568,0.026841,0.027466,0.023720,0.023720,0.023096,0.056804
1,2013,MTL,RENE BOURQUE,W,63,9.0,7.0,16.0,28.0,21.0,20.0,21.0,-1.0,109.0,398,433,-35,340,430,-90,15.0,90.0,32.0,13.0,9.0,54.0,894.0,0.010067,0.007830,0.017897,0.031320,0.023490,0.022371,0.023490,-0.001119,0.121924,0.445190,0.484340,-0.039150,0.380313,0.480984,-0.100671,0.016779,0.100671,0.035794,0.014541,0.010067,0.060403
2,2013,MTL,BRIAN GIONTA,W,81,18.0,22.0,40.0,50.0,47.0,38.0,39.0,-1.0,166.0,634,759,-125,541,652,-111,52.0,46.0,20.0,38.0,21.0,81.0,1446.0,0.012448,0.015214,0.027663,0.034578,0.032503,0.026279,0.026971,-0.000692,0.114799,0.438451,0.524896,-0.086445,0.374136,0.450899,-0.076763,0.035961,0.031812,0.013831,0.026279,0.014523,0.056017
3,2013,MTL,JOSH GORGES,D,66,1.0,13.0,14.0,44.0,55.0,43.0,37.0,6.0,34.0,527,736,-209,510,562,-52,175.0,97.0,12.0,11.0,29.0,37.0,1402.0,0.000713,0.009272,0.009986,0.031384,0.039230,0.030670,0.026391,0.004280,0.024251,0.375892,0.524964,-0.149073,0.363766,0.400856,-0.037090,0.124822,0.069187,0.008559,0.007846,0.020685,0.026391
4,2013,MTL,PK SUBBAN,D,82,10.0,43.0,53.0,111.0,79.0,70.0,74.0,-4.0,194.0,1115,920,195,792,882,-90,124.0,135.0,81.0,26.0,83.0,82.0,2019.0,0.004953,0.021298,0.026251,0.054978,0.039128,0.034671,0.036652,-0.001981,0.096087,0.552254,0.455671,0.096582,0.392273,0.436850,-0.044577,0.061417,0.066865,0.040119,0.012878,0.041109,0.040614


In [205]:
de.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/stats/2013_player_stats.csv', index='False', sep=',')
#de = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/stats/2013_player_stats.csv', index='False', sep=',')